In [1]:
import geopandas as gpd
import pandas as pd
import os

In [2]:
streams_716 = gpd.read_file('/Volumes/EB406_T7_2/source_streams/streams_716.gpkg')

In [3]:
streams_701 = gpd.read_file('/Volumes/EB406_T7_2/source_streams/streams_701.gpkg')

In [4]:
country_table = pd.read_parquet("s3://geoglows-v2/tables/v2-countries-table.parquet")

In [7]:
countries = ["Honduras"]

filtered_df = country_table[country_table["RiverCountry"].isin(countries)]
filtered_df

,LINKNO,lon,lat,RiverCountry,OutletCountry
5878930,710140399,-89.263111,14.547778,Honduras,El Salvador
5878931,710142383,-89.246222,14.535667,Honduras,El Salvador
5878941,710090799,-89.167222,14.525333,Honduras,El Salvador
5878970,710146351,-89.237222,14.526444,Honduras,El Salvador
5878975,710154288,-89.230222,14.516444,Honduras,El Salvador
...,...,...,...,...,...
6645054,770002951,-86.857333,13.372778,Honduras,Nicaragua
6645055,770008151,-86.704667,13.388000,Honduras,Nicaragua
6645066,770079911,-86.857333,13.372778,Honduras,Nicaragua
6645071,770086150,-86.869778,13.382000,Honduras,Nicaragua


In [6]:
# Subset the GeoDataFrame by matching LINKNO values
streams_701_subset = streams_701[streams_701['LINKNO'].isin(filtered_df['LINKNO'])]

# Check the result
print(streams_701_subset.head())


          LINKNO   DSLINKNO  strmOrder  USContArea  DSContArea TDXHydroRegion  \
11731  710090799  710100720          2  11651967.5  24292700.0     7020000010   
11742  710106672  710134450          2  13608080.5  25981688.0     7020000010   
11763  710108658  710124530          2  10063584.0  18348356.0     7020000010   
11770  710080883  710138418          2  10007754.5  12735672.0     7020000010   
11780  710094772  710096756          2  12328972.0  37407020.0     7020000010   

       VPUCode  TopologicalOrder  LengthGeodesicMeters  TerminalLink  musk_k  \
11731      701            205851           7266.700747     710078961   29067   
11742      701            205862           5018.297115     710078961   20073   
11763      701            205883           1248.997376     710078961    4996   
11770      701            205890           1064.321073     710078961    4257   
11780      701            205901           5158.718796     710078961   20635   

       musk_x                   

In [8]:
# Subset the GeoDataFrame by matching LINKNO values
streams_716_subset = streams_716[streams_716['LINKNO'].isin(filtered_df['LINKNO'])]

# Check the result
print(streams_716_subset.head())


         LINKNO   DSLINKNO  strmOrder  USContArea  DSContArea TDXHydroRegion  \
4451  770117261  770117260          2  10621312.5  38076848.0     7020047840   
4468  770117262  770138061          2  11373902.0  17940898.0     7020047840   
4488  770141183  770153661          2   9788268.5  31235104.0     7020047840   
4489  770142223  770194223          2   9987310.5  34388772.0     7020047840   
4490  770143263  770144303          2   9860901.5  18526852.0     7020047840   

      VPUCode  TopologicalOrder  LengthGeodesicMeters  TerminalLink  musk_k  \
4451      716            163433           6619.159044     770120380   26477   
4468      716            163454           7380.303779     770138061   29521   
4488      716            163476          12549.433445     770171341   50198   
4489      716            163477           2729.899843     770171341   10920   
4490      716            163478           2481.656870     770138061    9927   

      musk_x                                

In [5]:
countries = ["Honduras"]

filtered_df = country_table[country_table["RiverCountry"].isin(countries)]
filtered_df

,LINKNO,lon,lat,RiverCountry,OutletCountry
5878930,710140399,-89.263111,14.547778,Honduras,El Salvador
5878931,710142383,-89.246222,14.535667,Honduras,El Salvador
5878941,710090799,-89.167222,14.525333,Honduras,El Salvador
5878970,710146351,-89.237222,14.526444,Honduras,El Salvador
5878975,710154288,-89.230222,14.516444,Honduras,El Salvador
...,...,...,...,...,...
6645054,770002951,-86.857333,13.372778,Honduras,Nicaragua
6645055,770008151,-86.704667,13.388000,Honduras,Nicaragua
6645066,770079911,-86.857333,13.372778,Honduras,Nicaragua
6645071,770086150,-86.869778,13.382000,Honduras,Nicaragua


In [9]:
honduras_streams = pd.concat([streams_716_subset, streams_701_subset], ignore_index=True)


In [10]:
Honduras_Basins = gpd.read_file('/Users/rachel1/Downloads/Subcuencas/Cuencas Honduras/Cuencas_Honduras.shp')

In [ ]:
honduras_streams = honduras_streams.to_crs(Honduras_Basins.crs)
intersection = gpd.overlay(honduras_streams, Honduras_Basins, how='intersection')

In [ ]:
intersection

In [ ]:
highest_value_intersection = intersection.loc[intersection.groupby("Subbasin")["DSContArea"].idxmax()]

In [ ]:
highest_value_intersection.columns

In [14]:
# Drop columns (excluding 'geometry')
columns_to_drop = ['strmOrder', 'TDXHydroRegion', 'VPUCode', 'TopologicalOrder', 'LengthGeodesicMeters',
       'TerminalLink', 'musk_k', 'musk_x', 'OBJECTID', 'MAIN_BASIN',
       'Shape_Leng', 'Shape_Area', 'Basin_Name', 'Area_Km', '_mean', 'layer',
       'path']  # Replace with the actual column names you want to drop
highest_value_intersection = highest_value_intersection.drop(columns=columns_to_drop)


In [15]:
highest_value_intersection.to_file('/Users/rachel1/Downloads/basin_intersect.gpkg')